# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [10]:
print("Hey Guyzz !!")
!pip install ultralytics opencv-python-headless -q
!pip install dill

import cv2
import datetime
import os
import csv
import smtplib
import ssl
from email.message import EmailMessage
from IPython.display import Audio, display
from ultralytics import YOLO

# 📩 Email Alert Function
def send_email_alert(frame_path, event_type="fire"):
    sender_email = "santhoshk86278@gmail.com"
    receiver_email = "thulasidharanp.24mca@kongu.edu"
    app_password = "urha ftin wwno imxw"  # Store your app password in environment

    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    if event_type == "fire":
        subject = "🚨 URGENT: Fire & Human Presence Detected!"
        body = f"""
Dear Safety Team,

🔥 A potential FIRE has been detected in the monitored area.
👤 A HUMAN is also present, indicating a critical emergency.

📅 Timestamp: {timestamp}
📷 Attached Frame: Snapshot of incident.

Please verify immediately and alert authorities if necessary.

Stay safe,
🚒 Smart Surveillance System
        """
    else:
        subject = "🚗 ALERT: Accident Detected!"
        body = f"""
Dear Monitoring Team,

🚗 An ACCIDENT involving a vehicle (car, bike, etc.) has been detected in the surveillance feed.
👤 Human presence may also be involved.

📅 Timestamp: {timestamp}
📷 Attached Frame: Visual snapshot attached for review.

Kindly initiate appropriate safety protocols.

- Smart Detection System
        """

    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg.set_content(body)

    with open(frame_path, 'rb') as f:
        msg.add_attachment(f.read(), maintype='image', subtype='jpeg', filename=os.path.basename(frame_path))

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as server:
        server.login(sender_email, app_password)
        server.send_message(msg)
        print(f"📧 {event_type.capitalize()} alert email sent!")

# 🎯 Load Model
model = YOLO("best.pt")  # Use your custom trained model

# 📡 Connect to IP Webcam (LIVE STREAM)
ip_camera_url = 'http://192.168.245.128:8080/video'  # Replace with actual IP
cap = cv2.VideoCapture(ip_camera_url)

# 📂 Setup Output
frame_dir = "/content/detected_frames"
os.makedirs(frame_dir, exist_ok=True)

log_file = "/content/detection_log.csv"
with open(log_file, mode='w', newline='') as file:
    csv.writer(file).writerow(["Timestamp", "Fire", "Person", "Accident"])

frame_count = 0
fire_alert_triggered = False
accident_alert_triggered = False

print("🚀 Starting live detection...")

# 🔁 Frame-by-Frame Detection
while True:
    ret, frame = cap.read()
    if not ret:
        print("🚫 Unable to read from camera. Check IP or network.")
        break

    frame_count += 1
    results = model(frame)[0]
    classes = [model.names[int(cls)] for cls in results.boxes.cls]

    fire = 'fire' in classes
    person = 'person' in classes or 'human' in classes
    accident = any(label in classes for label in ['accident', 'car', 'bike'])

    # 🎨 Draw bounding boxes
    for box in results.boxes:
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        color = (0, 0, 255) if label.lower() == 'fire' else (0, 255, 0) if label.lower() == 'person' else (255, 0, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # 💾 Save frame
    frame_path = f"{frame_dir}/frame_{frame_count:04d}.jpg"
    cv2.imwrite(frame_path, frame)

    # 🧾 Log
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(log_file, mode='a', newline='') as file:
        csv.writer(file).writerow([timestamp, fire, person, accident])

    # 🔔 Send alerts
    if fire and not fire_alert_triggered:
        fire_alert_triggered = True
        print("🔥 Fire Detected!")
        display(Audio("poppop.ai - fire alarm siren.mp3", autoplay=True))  # Upload manually
        send_email_alert(frame_path, event_type="fire")

    if accident and not accident_alert_triggered:
        accident_alert_triggered = True
        print("💥 Accident Detected!")
        send_email_alert(frame_path, event_type="accident")

cap.release()
print("✅ Detection completed.")


Hey Guyzz !!
🚀 Starting live detection...

0: 384x640 (no detections), 276.6ms
Speed: 40.7ms preprocess, 276.6ms inference, 11.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 177.8ms
Speed: 4.2ms preprocess, 177.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 150.2ms
Speed: 4.3ms preprocess, 150.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 169.3ms
Speed: 3.3ms preprocess, 169.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 159.7ms
Speed: 3.3ms preprocess, 159.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 154.3ms
Speed: 2.6ms preprocess, 154.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 162.2ms
Speed: 3.3ms preprocess, 162.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640

📧 Fire alert email sent!

0: 384x640 (no detections), 178.9ms
Speed: 3.9ms preprocess, 178.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fire, 172.8ms
Speed: 2.3ms preprocess, 172.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 161.1ms
Speed: 3.5ms preprocess, 161.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fire, 126.1ms
Speed: 4.0ms preprocess, 126.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 108.3ms
Speed: 3.1ms preprocess, 108.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 118.8ms
Speed: 3.2ms preprocess, 118.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 138.4ms
Speed: 3.4ms preprocess, 138.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 121.2ms
Speed: 2.8m

KeyboardInterrupt: 